## CB02-4 Part Seven: Sovling Mnist

### 01 MLP

In [1]:
import torch

class mlp(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 16),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(16, 16),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(16, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x.reshape(-1, 28 * 28))
        return logits

### 02 dataset & dataloader

In [2]:
from torchvision import datasets
from torchvision import transforms
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform = transforms.ToTensor())

In [3]:
torch.manual_seed(0)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [4]:
## test the data
features, labels = next(iter(train_loader))
features.shape, labels.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

### 03 Training Loop

In [5]:
torch.manual_seed(0)
model = mlp(784, 10)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 1
for epoch in range(num_epochs):

    model.train() # set the model to training mode: redundant for this example

    for idx, (features, labels) in enumerate(train_loader):
        # clear the gradients for every batch
        optimizer.zero_grad()

        # forward pass
        logits = model(features)

        # compute the loss
        loss = torch.nn.functional.cross_entropy(logits, labels)
        
        # backward pass
        loss.backward()

        # update weights & biases through SGD
        optimizer.step()

        print(f'Epoch: {epoch}, Batch: {idx}, Loss: {loss:.2f}')
    
    model.eval() # set the model to evaluation mode: redundant for this example

Epoch: 0, Batch: 0, Loss: 2.28
Epoch: 0, Batch: 1, Loss: 2.27
Epoch: 0, Batch: 2, Loss: 2.24
Epoch: 0, Batch: 3, Loss: 2.23
Epoch: 0, Batch: 4, Loss: 2.19
Epoch: 0, Batch: 5, Loss: 2.16
Epoch: 0, Batch: 6, Loss: 2.01
Epoch: 0, Batch: 7, Loss: 2.21
Epoch: 0, Batch: 8, Loss: 2.20
Epoch: 0, Batch: 9, Loss: 2.05
Epoch: 0, Batch: 10, Loss: 1.99
Epoch: 0, Batch: 11, Loss: 1.85
Epoch: 0, Batch: 12, Loss: 2.52
Epoch: 0, Batch: 13, Loss: 2.19
Epoch: 0, Batch: 14, Loss: 2.18
Epoch: 0, Batch: 15, Loss: 2.06
Epoch: 0, Batch: 16, Loss: 1.91
Epoch: 0, Batch: 17, Loss: 1.83
Epoch: 0, Batch: 18, Loss: 1.66
Epoch: 0, Batch: 19, Loss: 1.54
Epoch: 0, Batch: 20, Loss: 2.00
Epoch: 0, Batch: 21, Loss: 1.89
Epoch: 0, Batch: 22, Loss: 1.88
Epoch: 0, Batch: 23, Loss: 2.20
Epoch: 0, Batch: 24, Loss: 1.75
Epoch: 0, Batch: 25, Loss: 2.07
Epoch: 0, Batch: 26, Loss: 1.61
Epoch: 0, Batch: 27, Loss: 1.60
Epoch: 0, Batch: 28, Loss: 1.53
Epoch: 0, Batch: 29, Loss: 1.45
Epoch: 0, Batch: 30, Loss: 1.40
Epoch: 0, Batch: 3

### 04 Evaluation

In [6]:
def eval(model, test_loader):
    model.eval()
    with torch.no_grad():
        num_correct = 0
        num_samples = 0
        for features, labels in test_loader:
            logits = model(features)
            _, predictions = logits.max(1)
            num_correct += (predictions == labels).sum()
            num_samples += predictions.size(0)
        print(f'Accuracy: {num_correct}/{num_samples} = {num_correct/num_samples:.2f}')

eval(model, test_loader)

Accuracy: 9290/10000 = 0.93


### 05 Model Save

In [7]:
torch.save(model.state_dict(), 'model_mnist.pth')

In [8]:
model = mlp(784, 10)
model.load_state_dict(torch.load('model_mnist.pth'))
eval(model, test_loader)

Accuracy: 9290/10000 = 0.93
